In [4]:
import pandas as pd
df = pd.read_excel(r"C:\Users\ASUS\Desktop\python\DigitalX Project\handeled_missing.xlsx")

In [6]:
df.columns

Index(['business_service', 'category', 'priority', 'sys_created_on',
       'resolved_at', 'assigned_to', 'state', 'cmdb_ci', 'caller_id',
       'short_description', 'assignment_group', 'close_code', 'close_notes'],
      dtype='object')

In [10]:
df['day'] = df['sys_created_on'].dt.day
df['month'] = df['sys_created_on'].dt.month
df['year'] = df['sys_created_on'].dt.year

In [11]:
df

,business_service,category,priority,sys_created_on,resolved_at,assigned_to,state,cmdb_ci,caller_id,short_description,assignment_group,close_code,close_notes,day,month,year
0,GRP56.34 PRXX (DigitalC Workplace management),Business Service,3 - Moderate,2018-10-03 00:00:36,2018-10-03 10:00:39,sn int-test-svc Functional,Closed,GRP56.34 PRXX (DigitalC Workplace management),Event Management,Monitor status is error for ukwwnppcm0981_[PCM...,EngineeringSystems-Lev2,3rd Party Issue,Covered in Problem management.,3,10,2018
1,GRP56.34 PRXX (DigitalC Workplace management),Business Service,3 - Moderate,2018-10-03 00:00:36,2018-10-03 10:00:39,Emily White,Closed,GRP56.34 PRXX (DigitalC Workplace management),Event Management,Monitor status is error for ukwwnppcm0981_[PCM...,EngineeringSystems-Lev2,3rd Party Issue,Covered in Problem management.,3,10,2018
2,NxGen Tech. Business SErvices - Backup,Hardware-Infrastructure,3 - Moderate,2018-10-03 00:01:38,2018-10-03 00:16:32,prd-sql Functional,Closed,ERP SFA Basis Non-PRXX Support,Event Management,IND01PXXRBK01: NBR: OracleDb:Backup : Status: ...,Bakup_Tek-SVC-Lev2,Other not on list,Resolving from CliqApps,3,10,2018
3,NxGen Tech. Business SErvices - Backup,Hardware-Infrastructure,3 - Moderate,2018-10-03 00:01:38,2018-10-03 00:16:32,Grace Harris,Closed,ERP SFA Basis Non-PRXX Support,Event Management,IND01PXXRBK01: NBR: OracleDb:Backup : Status: ...,Bakup_Tek-SVC-Lev2,Other not on list,Resolving from CliqApps,3,10,2018
4,NxGen Tech. Business SErvices - Backup,Hardware-Infrastructure,3 - Moderate,2018-10-03 00:21:33,2018-10-03 01:36:27,Jack Mitchell,Closed,ERP SFA Basis Non-PRXX Support,Event Management,IND01PXXRBK01: SLQ: OracleDb:Backup : Status: ...,Bakup_Tek-SVC-Lev2,Other not on list,Resolving from CliqApps,3,10,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4948,L1/LO L3 Support,Software,3 - Moderate,2018-10-04 10:20:45,2018-10-05 20:50:54,Zoe Green,Closed,Process Reorder Service,ITSM-PO Functional,Error reported by alert rule 'AR_C142a',SFA-PI-PO-Lev2,Network Issue,Closure - Alerts triggered due to CHIP downtim...,4,10,2018
4949,L1/LO L3 Support,Software,3 - Moderate,2018-10-04 10:20:45,2018-10-05 20:50:54,Jack Green,Closed,Process Reorder Service,ITSM-PO Functional,Error reported by alert rule 'AR_C142a',SFA-PI-PO-Lev2,Network Issue,Closure - Alerts triggered due to CHIP downtim...,4,10,2018
4950,L1/LO L3 Support,Software,3 - Moderate,2018-10-04 10:20:45,2018-10-06 08:02:08,Samuel Young,Closed,Process Reorder Service,ITSM-PO Functional,Error reported by alert rule 'AR_C142d',SFA-PI-PO-Lev2,Network Issue,_x000D_\nAlerts triggered due to CHIP downtime...,4,10,2018
4951,L1/LO L3 Support,Software,3 - Moderate,2018-10-04 10:20:45,2018-10-06 08:02:08,Ruby White,Closed,Process Reorder Service,ITSM-PO Functional,Error reported by alert rule 'AR_C142d',SFA-PI-PO-Lev2,Network Issue,_x000D_\nAlerts triggered due to CHIP downtime...,4,10,2018


In [12]:
# Calculate resolution time in hours
df["resolution_time"] = (df["resolved_at"] - df["sys_created_on"]).dt.total_seconds() / 3600

#Average resolution time per Category and Priority
avg_resolution_time = df.groupby(["category", "priority"])['resolution_time'].mean().reset_index()

#Ticket closure rate per Assigned Group
def closure_rate_func(x):
    return (x == "Closed").sum() / len(x)

closure_rate = df.groupby("assignment_group")["state"].apply(closure_rate_func).reset_index()
closure_rate.columns = ["assignment_group", "closure_rate"]

#Monthly Ticket Summary
df["month_year"] = df["sys_created_on"].dt.to_period("M")
monthly_summary = df.groupby("month_year").agg(
    total_tickets=("category", "count"),
    avg_resolution_time=("resolution_time", "mean"),
    closure_rate=("state", closure_rate_func)
).reset_index()

#Display results
print("Average Resolution Time per Category and Priority:\n", avg_resolution_time.head())
print("\nTicket Closure Rate per Assigned Group:\n", closure_rate.head())
print("\nMonthly Ticket Summary:\n", monthly_summary.head())

Average Resolution Time per Category and Priority:
            category      priority  resolution_time
0  Business Service  3 - Moderate        12.199246
1     Collaboration       4 - Low         1.280889
2        Facilities       4 - Low        17.371528
3           Finance  1 - Critical        53.048611
4           Finance      2 - High         3.592778

Ticket Closure Rate per Assigned Group:
              assignment_group  closure_rate
0  ABV-SN-Ecrystal-IMTransfer      1.000000
1       ADInfrastructure-Lev2      0.666667
2        Access-Governance-L3      1.000000
3      Access-Governance-Lev2      1.000000
4             AdobeCloud-Lev2      1.000000

Monthly Ticket Summary:
   month_year  total_tickets  avg_resolution_time  closure_rate
0    2018-10           4953            67.883884      0.977186
